In [1]:
! pip install torch torchvision opencv-python


Defaulting to user installation because normal site-packages is not writeable


In [9]:
import torch
import torch.nn as nn

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ResNet9, self).__init__()
        # Example convolutional layers
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        
        # Define the fully connected (fc) layer
        # Assuming the input to fc is flattened from a 128x32x32 tensor
        self.fc = nn.Linear(128 * 32 * 32, num_classes)  # Adjust dimensions based on your architecture
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)  # Fully connected layer
        return x



In [13]:
# Example to calculate dimensions
dummy_input = torch.randn(1, 3, 32, 32)  # Replace with your input shape
model = ResNet9(in_channels=3, num_classes=10)

# Pass dummy input through convolutional layers only
with torch.no_grad():
    features = model.conv1(dummy_input)
    features = model.conv2(features)
    print(features.shape)  # Inspect the shape


torch.Size([1, 128, 32, 32])


In [14]:
# Define the model
model = ResNet9(in_channels=3, num_classes=10)

# Example input tensor
input_tensor = torch.randn(1, 3, 32, 32)  # Adjust shape as needed

# Perform inference
with torch.no_grad():
    output = model(input_tensor)
    predicted_class = output.argmax(1).item()

print(f'Predicted Class: {predicted_class}')


Predicted Class: 4


In [10]:
import torch

# Load the full model
model = torch.load('models/plant-disease-model-complete.pth', map_location='cpu')

# Set the model to evaluation mode
model.eval()

print(model)  # Inspect the loaded model



ResNet9(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

/tmp/ipykernel_277423/3847535290.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('models/plant-disease-model-complete.pth', map_location='cpu')


In [11]:
# Define preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model's input size
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Load image
    return transform(image).unsqueeze(0)  # Add batch dimension


In [12]:
# Load and preprocess an image
image_path = 'images_for_test/5c1b9970-0fc5-4aaf-8ae4-3255aa993630.jpeg'  # Replace with your image path
input_tensor = preprocess_image(image_path)

# Perform inference
with torch.no_grad():
    output = model(input_tensor)
    predicted_class = output.argmax(1).item()

print(f'Predicted Class: {predicted_class}')


AttributeError: 'ResNet9' object has no attribute 'fc'

In [51]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import cv2

# Define the ResNet9 model or import it if defined in another file
class ResNet9(torch.nn.Module):
    def __init__(self, num_classes):
        super(ResNet9, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(inplace=True)
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # AdaptiveAvgPool2d ensures that spatial dimensions are always (1, 1)
        self.classifier = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d((1, 1)),
            torch.nn.Flatten(),
            torch.nn.Linear(128, num_classes)  # Match the output size of the conv layers
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.classifier(x)
        print("the classifier output :" , x.shape)
        return x

# Reload the model with the updated definition
num_classes = 5  # Update based on the dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet9(num_classes=num_classes)
model = torch.load("models/plant-disease-model-complete.pth", map_location=device)  # Load the entire model object
model.to(device)
model.eval()  # Set the model to evaluation mode

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a more reasonable size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Preprocess the input image
def preprocess_image(image):
    image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
    return transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device

# Predict for a single image
def predict_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Image not found at {image_path}")

        preprocessed_image = preprocess_image(image)
        with torch.no_grad():
            predictions = model(preprocessed_image)
            if isinstance(predictions, torch.Tensor):
                predicted_class = torch.argmax(predictions, dim=1).item()
            else:
                raise ValueError("Model output is not a tensor. Check model implementation.")

        return predicted_class, predictions
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None, None

# Example usage
image_path = "images_for_test/cfb73761-e9b0-49de-bfa0-c24fa980f5b2.jpeg"

# Predict on an image
predicted_class, predictions = predict_image(image_path)
if predicted_class is not None:
    print(f"Predicted Class: {predicted_class}")
    print(f"Predictions: {predictions}")


Error during prediction: mat1 and mat2 shapes cannot be multiplied (1x25088 and 512x38)


/tmp/ipykernel_286924/3346857973.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("models/plant-disease-model-complete.pth", map_location=device)  # L

In [26]:
print("Expected input size:", model.conv1[0].in_channels)  # إذا كنت تستخدم Conv2d


Expected input size: 3


In [46]:
predicted_class, predictions = predict_image(image_path)
if predicted_class is not None:
    print(f"Predicted Class: {predicted_class}")
    print(f"Predictions: {predictions}")
    

Error during prediction: a Tensor with 3136 elements cannot be converted to Scalar


In [67]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from PIL import Image
import cv2



class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, pool=False):
        super().__init__()
        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        ]
        if pool:
            layers.append(nn.MaxPool2d(2))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_diseases):
        super().__init__()
        self.conv1 = ConvBlock(in_channels, 64)
        self.conv2 = ConvBlock(64, 128, pool=True)  # out_dim: 128 x 64 x 64
        self.res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128))
        self.conv3 = ConvBlock(128, 256, pool=True)  # out_dim: 256 x 16 x 16
        self.conv4 = ConvBlock(256, 512, pool=True)  # out_dim: 512 x 4 x 4
        self.res2 = nn.Sequential(ConvBlock(512, 512), ConvBlock(512, 512))
        self.classifier = nn.Sequential(
            nn.MaxPool2d(4),
            nn.Flatten(),
            nn.Linear(512, num_diseases)
        )

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out


# Reload the model with the updated definition
num_classes = 38   # Update based on the dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet9(in_channels=3,num_diseases=num_classes)
model = torch.load("models/plant-disease-model-complete.pth", map_location=device)  # Load the entire model object
model.to(device)
model.eval()  # Set the model to evaluation mode

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to a more reasonable size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Preprocess the input image
def preprocess_image(image):
    image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
    return transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device

class_labels = ['Tomato___Late_blight', 'Tomato___healthy', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Potato___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Tomato___Early_blight', 'Tomato___Septoria_leaf_spot', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Strawberry___Leaf_scorch', 'Peach___healthy', 'Apple___Apple_scab', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Bacterial_spot', 'Apple___Black_rot', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Peach___Bacterial_spot', 'Apple___Cedar_apple_rust', 'Tomato___Target_Spot', 'Pepper,_bell___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Potato___Late_blight', 'Tomato___Tomato_mosaic_virus', 'Strawberry___healthy', 'Apple___healthy', 'Grape___Black_rot', 'Potato___Early_blight', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Common_rust_', 'Grape___Esca_(Black_Measles)', 'Raspberry___healthy', 'Tomato___Leaf_Mold', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Pepper,_bell___Bacterial_spot', 'Corn_(maize)___healthy']  # Replace with your actual labels

# Function to annotate the image with the predicted class
def annotate_image(image_path, predicted_class):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found at {image_path}")
    
    # Add a rectangle (bounding box) - Adjust coordinates as needed
    height, width, _ = image.shape
    start_point = (10, 10)  # Top-left corner of the bounding box
    end_point = (width - 10, height - 10)  # Bottom-right corner of the bounding box
    color = (0, 255, 0)  # Green color for the bounding box
    thickness = 2  # Thickness of the rectangle
    
    cv2.rectangle(image, start_point, end_point, color, thickness)
    
    # Add the predicted label text
    label = f"Predicted: {class_labels[predicted_class]}"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    text_color = (0, 255, 0)
    text_thickness = 2
    
    cv2.putText(image, label, (20, 50), font, font_scale, text_color, text_thickness)
    
    # Save the annotated image
    annotated_path = "annotated_image.jpg"
    cv2.imwrite(annotated_path, image)
    print(f"Annotated image saved to {annotated_path}")
    
    return annotated_path

# Predict for a single image
def predict_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Image not found at {image_path}")

        preprocessed_image = preprocess_image(image)
        with torch.no_grad():
            predictions = model(preprocessed_image)
            if isinstance(predictions, torch.Tensor):
                predicted_class = torch.argmax(predictions, dim=1).item()
            else:
                raise ValueError("Model output is not a tensor. Check model implementation.")

        # Annotate the image
        annotated_image_path = annotate_image(image_path, predicted_class)
        
        return predicted_class, predictions, annotated_image_path
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None, None, None

# Example usage
image_path = "images_for_test/Lavender-with-White-Spots.png"

# Predict and annotate the image
predicted_class, predictions, annotated_image_path = predict_image(image_path)
if predicted_class is not None:
    print(f"Predicted Class: {class_labels[predicted_class]}")  # Display the class label
    print(f"Predictions: {predictions}")  # Display the raw predictions
    print(f"Annotated Image Path: {annotated_image_path}")

Annotated image saved to annotated_image.jpg
Predicted Class: Tomato___healthy
Predictions: tensor([[ -77.8094,   -0.1820, -108.9794, -131.8778, -104.7580, -111.1996,
          -95.5578, -109.2154,  -50.7834, -155.1822, -107.0426,  -88.8749,
          -89.5843, -114.3271, -104.8174, -171.8780,  -81.2308,  -48.2699,
          -13.6952,  -12.1107,  -98.8678, -161.6070, -117.0350,  -90.4716,
         -130.2487, -168.7204, -110.2874, -126.8831, -129.9841,  -31.6615,
          -31.9136, -108.0810,  -42.1047, -136.2311,  -70.8965,  -89.1100,
          -35.6075,  -96.8336]], device='cuda:0')
Annotated Image Path: annotated_image.jpg


/tmp/ipykernel_286924/519424502.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("models/plant-disease-model-complete.pth", map_location=device)  # Lo